# Assigment 6
**Edit by LJL@PKU**

### 22275：二叉搜索树的遍历

In [3]:
def preorder_to_postorder(preorder):
    if not preorder:
        return []
    
    root = preorder[0]
    left_preorder = [x for x in preorder[1:] if x < root]
    right_preorder = [x for x in preorder[1:] if x > root]
    
    left_postorder = preorder_to_postorder(left_preorder)
    right_postorder = preorder_to_postorder(right_preorder)
    
    return left_postorder + right_postorder + [root]

n = int(input())
preorder = list(map(int, input().split()))
postorder = preorder_to_postorder(preorder)
print(' '.join(map(str, postorder)))


1 3 2 5 4


### 05455:二叉搜索树的层次遍历

In [6]:
from collections import deque

numbers = list(map(int, input().strip().split()))
numbers = list(dict.fromkeys(numbers)) # remove duplicates

class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

# 永远从根开始往下插入，遇到比当前根节点值小的往左插，否则往右插
def insert(node, value):
    if node is None:
        return TreeNode(value)
    if value < node.val:
        node.left = insert(node.left, value)
    else:
        node.right = insert(node.right, value)
    return node

def traversal(root):
    queue = deque([root])
    travesal = []
    while queue:
        node = queue.popleft()
        travesal.append(node.val)
        if node.left:
            queue.append(node.left)
        if node.right:
            queue.append(node.right)
    return travesal

root = None
for number in numbers:
    root = insert(root, number)
traversal = traversal(root)
print(' '.join(map(str, traversal)))

51 45 59 4 50 86 2 15 76 97 1 13 20 60 77 90 11 14 19 30 61 82 96 7 29 37 62 79 91 6 33 39 74 34 72


### 04078：实现堆结构

In [9]:
class Heap():
    def __init__(self, array = [], less = lambda x,y:x<y):
        self._a = array[:]
        self._size = len(array)
        self._less = less
        self.makeHeap()
    
    def top(self):
        return self._a[0]
    
    def pop(self):
        tmp = self._a[0]
        self._a[0] = self._a[-1]
        self._a.pop()
        self._size -= 1
        self._goDown(0)
        return tmp
    
    def append(self, x):
        self._size +=1
        self._a.append(x)
        self._goUp(self._size - 1)
    
    def _goUp(self, i):
        if i == 0:
            return
        f = (i-1)//2
        if self._less(self._a[i], self._a[f]):
            self._a[i], self._a[f] = self._a[f], self._a[i]
            self._goUp(f)

    def _goDown(self,i):
        if i*2 + 1 >= self._size:
            return
        L, R = i*2 + 1, i*2+2
        if R >= self._size or self._less(self._a[L], self._a[R]):
            s = L
        else:
            s = R
        
        if self._less(self._a[s], self._a[i]):
            self._a[i], self._a[s] = self._a[s], self._a[i]
            self._goDown(s)
    
    def makeHeap(self):
        i = (self._size - 1 - 1)// 2
        for k in range(i, -1, -1):
            self._goDown(k)

    def heapSort(self):
        for i in range(self._size-1, -1, -1):
            self._a[i], self._a[0] = self._a[0], self._a[i]
            self._size -= 1
            self._goDown(0)
        
        self._a.reverse()
        return self._a


n = int(input())
BinHeap = Heap()
for _ in range(n):
    op = list(map(int,input().split()))
    if op[0] == 1:
        BinHeap.append(op[1])
    
    if op[0] == 2:
        num = BinHeap.pop()
        print(num)


1


### 22161：哈夫曼编码树

In [ ]:
import heapq

class Node:
    def __init__(self, weight, char=None):
        self.weight = weight
        self.char = char
        self.left = None
        self.right = None

    # 如果两个Node对象的权重相同，则按照它们的字符顺序进行比较。这意味着如果两个节点的权重相同，则会根据它们的字符（char属性）的字典顺序进行排序。
    def __lt__(self, other):
        if self.weight == other.weight:
            return self.char < other.char
        return self.weight < other.weight

def build_huffman_tree(characters):
    heap = []

    # 将结点加入到堆中，保持堆的不变性。
    for char, weight in characters.items():
        heapq.heappush(heap, Node(weight, char))

    # 合并堆，直至只剩一个
    while len(heap) > 1:

        # 弹出并返回 heap 的最小的两个元素，在本题中即为出现次数最少的两个元素，并保持堆的不变性。
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        # merged = Node(left.weight + right.weight) #note: 合并后，char 字段默认值是空
        # 合并两个最小的元素，算出其父结点的值，字符值用左结点和右结点中小的代替（为什么？）
        merged = Node(left.weight + right.weight, min(left.char, right.char)) 
        merged.left = left
        merged.right = right
        heapq.heappush(heap, merged)

    return heap[0]

# 对哈夫曼树的结点进行编码，用递归对每个结点值进行编码
def encode_huffman_tree(root):
    codes = {}

    def traverse(node, code):
        # if node.char: 可能和上面改动的地方对应
        # 如果左右子结点均为None，编码即为自身
        if node.left is None and node.right is None:
            codes[node.char] = code
        # 左结点赋0，右结点赋1
        else:
            traverse(node.left, code + '0')
            traverse(node.right, code + '1')

    # 从根结点开始编码
    traverse(root, '')
    return codes

# 对输入的字符进行哈夫曼编码，直接索引字典即可
def huffman_encoding(codes, string):
    encoded = ''
    for char in string:
        encoded += codes[char]
    return encoded

# 对输入的哈夫曼编码进行解码
def huffman_decoding(root, encoded_string):
    decoded = ''
    node = root
    for bit in encoded_string:
        if bit == '0':
            node = node.left
        else:
            node = node.right

        # 只有左右子结点的值均为None时也就是无叶子结点时，才会把解码的字符添加到解码字符串中
        # if node.char:
        if node.left is None and node.right is None:
            decoded += node.char
            node = root
    return decoded

# 读取输入
n = int(input())
characters = {}
for _ in range(n):
    char, weight = input().split()
    characters[char] = int(weight)

#string = input().strip()
#encoded_string = input().strip()

# 构建哈夫曼编码树
huffman_tree = build_huffman_tree(characters)

# 编码和解码
codes = encode_huffman_tree(huffman_tree)

strings = []
while True:
    try:
        line = input()
        strings.append(line)

    except EOFError:
        break

results = []
#print(strings)
for string in strings:
    # 如果输入的是哈夫曼编码
    if string[0] in ('0','1'):
        results.append(huffman_decoding(huffman_tree, string))
    # 如果输入的是哈夫曼字符
    else:
        results.append(huffman_encoding(codes, string))

for result in results:
    print(result)

### 平衡二叉树的建立

In [21]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def __init__(self):
        self.root = None

    def insert(self, root, val):
        if not root:
            return TreeNode(val)
        elif val < root.val:
            root.left = self.insert(root.left, val)
        else:
            root.right = self.insert(root.right, val)
        
        root.height = 1 + max(self.getHeight(root.left), self.getHeight(root.right))
        
        balance = self.getBalance(root)
        
        # Left Left Case
        if balance > 1 and val < root.left.val:
            return self.rightRotate(root)
        
        # Right Right Case
        if balance < -1 and val > root.right.val:
            return self.leftRotate(root)
        
        # Left Right Case
        if balance > 1 and val > root.left.val:
            root.left = self.leftRotate(root.left)
            return self.rightRotate(root)
        
        # Right Left Case
        if balance < -1 and val < root.right.val:
            root.right = self.rightRotate(root.right)
            return self.leftRotate(root)
        
        return root

    def getHeight(self, root):
        if not root:
            return 0
        return root.height

    def getBalance(self, root):
        if not root:
            return 0
        return self.getHeight(root.left) - self.getHeight(root.right)

    # 左旋操作，其中z为失衡结点，y为失衡结点的左子树，T2为失衡结点的左孩
    # 右旋就是把失衡结点z连给y的左子结点，把T2连接给失衡结点z的右结点    
    def leftRotate(self, z):
        y = z.right
        T2 = y.left
        
        y.left = z
        z.right = T2
        
        z.height = 1 + max(self.getHeight(z.left), self.getHeight(z.right))
        y.height = 1 + max(self.getHeight(y.left), self.getHeight(y.right))
        
        return y

    # 右旋操作，其中y为失衡结点，x为失衡结点的左子树，T2为失衡结点的右孩
    # 右旋就是把y连给x的右结点，把T2连接给失衡结点y的左结点
    def rightRotate(self, y):
        x = y.left
        T2 = x.right
        
        x.right = y
        y.left = T2
        
        y.height = 1 + max(self.getHeight(y.left), self.getHeight(y.right))
        x.height = 1 + max(self.getHeight(x.left), self.getHeight(x.right))
        
        return x

    def preOrderTraversal(self, root):
        result = []
        if root:
            result.append(root.val)
            result += self.preOrderTraversal(root.left)
            result += self.preOrderTraversal(root.right)
        return result

# 输入
n = int(input())
elements = list(map(int, input().split()))

# 构建 AVL 树
avl_tree = AVLTree()
for element in elements:
    avl_tree.root = avl_tree.insert(avl_tree.root, element)

# 输出先序遍历序列
pre_order_result = avl_tree.preOrderTraversal(avl_tree.root)
# for val in pre_order_result:
#     print(val, end=" ")
print(' '.join(str(val) for val in pre_order_result))


3 2 6 5 8


### 02524：宗教信仰

In [18]:
counter = 0
while True:
    counter +=1
    n, m = map(int, input().split())
    if n == 0 and m == 0:
        break

    num = 0
    religion = [-1]*n
    for _ in range(m):
        i, j = map(int, input().split())
        if religion[i-1] != -1:
            religion[j-1] = religion[i-1]
        elif religion[j-1] != -1:
            religion[i-1] = religion[j-1]
        else:
            num +=1
            religion[i-1] = num
            religion[j-1] = num
    
    for _ in range(n):
        if religion[_]== -1:
            num += 1
    print('Case {}:'.format(counter), num)



Case 1: 1
Case 2: 7
